In [1]:
from array import array
import P201_Functions as p201
import numpy as np
import math

p201.set_dark_mode(True)


ModuleNotFoundError: No module named 'P201_Functions'

In [ ]:
# Initial Conditions
length = 9.799 # length of pendulum in meters
method = 2 # 1 = Euler, 2 = Verlet
theta0 = -7.82*np.pi/180.0 # initial angle in radians
omega = 0

theta = theta0

# Other constants
gravity = 9.799 # gravitional acceleration in m/s^2
a = 0.0 # forward acceleration of van
g_over_L = gravity/length
a_over_L = a/length

tau = 0.00001 # timestep in seconds
nStep = int(1/tau*102.27)

Pi = math.pi

In [ ]:
#accel = -g_over_L*math.sin(theta) #simple pendulum
accel = -g_over_L*math.sin(theta) - a_over_L*math.cos(theta) #pendulum in acc. van
theta_old = theta - omega*tau + 0.5*tau*tau*accel
print (accel, theta, theta_old)

In [ ]:
t_plot = array('d')
th_plot = array('d')
th_plot_corrected = array('d')
period = array('d')
trev = array('d')
period_corrected = array('d')
trev_corrected = array('d')


In [ ]:

time = 0.0
irev = 0.0
dsum = 0.0

for iStep in range(1,nStep+1):
    t_plot.append(time)
    th_plot.append(theta*180/Pi)
    dsum += theta*180/Pi
    time = time + tau
    #accel = -g_over_L*math.sin(theta) #simple pendulum
    accel = -g_over_L*math.sin(theta) - a_over_L*math.cos(theta) #pendulum in acc. van
    
    if (method == 1):
        theta_old = theta
        theta = theta + tau*omega
        omega = omega + tau*accel
    else:
        theta_new = 2*theta - theta_old + tau*tau*accel
        theta_old = theta
        theta = theta_new
    
    if (theta*theta_old < 0):
        print ("Zero Crossing at time t = %f" % time)
        if (irev == 0):
            time_old = time
        else:
            period.append(2*(time - time_old))
            trev.append(time)
            time_old = time
        irev = irev + 1

theta_avg = dsum/nStep
        
for iStep in range(1,nStep+1):
    th_plot_corrected.append(th_plot[iStep-1]-theta_avg)

time = 0.0
irev = 0.0
dsum = 0.0
time_old = 0.0

for iStep in range(1,nStep-1):
    time = time + tau
    if (th_plot_corrected[iStep+1]*th_plot_corrected[iStep] < 0):
        print ("Zero Crossing at time t = %f" % time)
        if (irev==0):
            time_old = time
        else:
            period_corrected.append(2*(time - time_old))
            trev_corrected.append(time)
            time_old = time
        irev = irev + 1

In [ ]:
theta_avg_theory = np.arctan(-a/gravity)*180.0/np.pi
nPeriod = int(irev - 1)
print ("Equilibrium Angle = %11.7f" % theta_avg)
print ("Equilibrium Angle Theory = %11.7f" % theta_avg_theory)
print ("Number of periods = ",nPeriod)

In [ ]:
from ROOT import TCanvas
from ROOT import TGraph, TGraphErrors
from ROOT import gStyle

xcanvas = 1000
ycanvas = 800

c1 = TCanvas( 'c1', 'Pendulum Amplitude vs. Time', 0, 0, xcanvas, ycanvas )
c1.SetGridx()
c1.SetGridy()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderMode(-1 )
c1.GetFrame().SetBorderSize( 5 )

c2 = TCanvas( 'c2', 'Pendulum Period vs. Time', 0, 0, xcanvas, ycanvas )
c2.SetGridx()
c2.SetGridy()
c2.GetFrame().SetFillColor( 21 )
c2.GetFrame().SetBorderMode(-1 )
c2.GetFrame().SetBorderSize( 5 )

gr = TGraph(nStep,t_plot,th_plot_corrected)
grr = TGraph(nPeriod,trev_corrected,period_corrected)

In [ ]:
gr.SetMarkerColor(4)
grr.SetMarkerColor(4)
gr.SetMarkerStyle(21)
grr.SetMarkerStyle(22)
gr.SetTitle("Pendulum Amplitude vs. Time")
gr.GetXaxis().SetTitle("Time")
gr.GetYaxis().SetTitle("Amplitude")
grr.SetTitle("Pendulum Period vs. Time")
grr.GetXaxis().SetTitle("Time")
grr.GetYaxis().SetTitle("Period")

c1.cd()
gr.Draw("AP")
c1.Draw()

In [ ]:
c2.cd()
grr.Draw("AP")
c2.Draw()

In [ ]:
AvePeriod = 0.0
ErrorBar = 0.0
for i in range(1,nPeriod+1):
    AvePeriod = AvePeriod + period_corrected[i-1]
AvePeriod = AvePeriod/nPeriod
for i in range(1,nPeriod+1):
    ErrorBar = ErrorBar + (period_corrected[i-1]-AvePeriod)*(period_corrected[i-1]-AvePeriod)
ErrorBar = math.sqrt(ErrorBar/(nPeriod*(nPeriod-1)))
print("Average Period = %f +/- %f" % (AvePeriod,ErrorBar))
    

In [ ]:
t0=theta0
#t_small_angle = 2.0*Pi/math.sqrt(g_over_L)
t_small_angle = 2.0*Pi/math.sqrt(math.sqrt(gravity**2+a**2)/length)
#t_infinite = 2.0*Pi/math.sqrt(g_over_L)*(1.0+t0*t0/16.0+math.pow(t0,4)*11.0/3072.0+math.pow(t0,6)*173.0/737280.0)
t_infinite = t_small_angle*(1.0+t0*t0/16.0+math.pow(t0,4)*11.0/3072.0+math.pow(t0,6)*173.0/737280.0)
#error_infinite = 2.0*Pi/math.sqrt(g_over_L)*math.pow(t0,8)*22931.0/1321205760.0
error_infinite = t_small_angle*math.pow(t0,8)*22931.0/1321205760.0

In [ ]:
print("Small Angle prediction = %f" % (t_small_angle))
print("Infinite series prediction = %f +/- %f" % (t_infinite,error_infinite))

In [ ]:
x = array('d')
y = array('d')
ex = array('d')
ey = array('d')
npoints = 3
x.append(1)
x.append(2)
x.append(3)
ex.append(0)
ex.append(0)
ex.append(0)
y.append(AvePeriod)
y.append(t_infinite)
y.append(t_small_angle)
ey.append(ErrorBar)
ey.append(error_infinite)
ey.append(0)

c3 = TCanvas("c3","c3",100,0,800,600)
gr3 = TGraphErrors(npoints,x,y,ex,ey)
gr3.SetMarkerStyle(20)
gr3.SetTitle("Theory vs. Simulation")
gr3.GetXaxis().SetTitle("1=Sim,2=Theory,3=SmallAngle")
gr3.GetYaxis().SetTitle("Average Period (s)")
gr3.Draw("AP")
c3.Draw()